<a href="https://colab.research.google.com/github/v-vipriy/data-eng-tech-assessment/blob/master/script/Data_Eng_Tech_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone 'https://github.com/v-vipriy/data-eng-tech-assessment/'

fatal: destination path 'data-eng-tech-assessment' already exists and is not an empty directory.


In [ ]:
try:
  from apache_beam.options.pipeline_options import PipelineOptions
  import   apache_beam as beam
  import urllib.request
  import json
except:
  ! pip   install apache-beam[interactive]
  from apache_beam.options.pipeline_options import PipelineOptions
  import   apache_beam as beam
  import urllib.request
  import json

In [ ]:
class load_json_as_dictionary(beam.DoFn):
  def process(self, element):
    result = [{k: int(v) if k == 'locationid' else v for k, v in d.items()} for d in map(lambda x: dict(x), json.loads(element))]
    return result

class select_elements_location_data(beam.DoFn):
  def process(self, element):
    result = [{k: v for k, v in element.items() if k in ('location_id', 'sensor_description','sensor_name')}]
    return result

class drop_redundant_fields(beam.DoFn):
  def process(self, element):
    result = [{k: v for k, v in element.items() if k not in ('location_id')}]
    return result

class LeftJoin(beam.PTransform):
    """This PTransform performs a left join given source_pipeline_name, source_data,
     join_pipeline_name, join_data, common_key constructors"""

    def __init__(self, source_pipeline_name, source_data, join_pipeline_name, join_data, left_key,right_key):
        self.join_pipeline_name = join_pipeline_name
        self.source_data = source_data
        self.source_pipeline_name = source_pipeline_name
        self.join_data = join_data
        self.left_key = left_key
        self.right_key = right_key
        # print("Inside LeftJoin -> __init))")

    def expand(self, pcolls):
        # print("Inside LeftJoin -> expand : 1")
        def _format_as_common_key_tuple(data_dict, left_key,right_key):
            # print("Inside LeftJoin -> expand : map(data_dict[common_key],int)")
            # print(type(data_dict), type(common_key))
            # print(data_dict)
            try:
              # print("data_dict[left_key] ", data_dict[left_key])
              return data_dict[left_key], data_dict
            except:
              # print("data_dict[right_key] ", data_dict[right_key])
              return data_dict[right_key], data_dict

        # print("Inside LeftJoin -> expand : 2")
        return ({pipeline_name: pcoll | 'Convert to ({0}/{1}, object) for {2}'
                .format(self.left_key,self.right_key, pipeline_name)
                                >> beam.Map(_format_as_common_key_tuple, self.left_key, self.right_key)
                 for (pipeline_name, pcoll) in pcolls.items()}
                | 'CoGroupByKey {0}'.format(pcolls.keys()) >> beam.CoGroupByKey()
                | 'Unnest Cogrouped' >> beam.ParDo(UnnestCoGrouped(),
                                                   self.source_pipeline_name,
                                                   self.join_pipeline_name)
                )

class UnnestCoGrouped(beam.DoFn):
    """This DoFn class unnests the CogroupBykey output and emits """

    def process(self, input_element, source_pipeline_name, join_pipeline_name):
        group_key, grouped_dict = input_element
        join_dictionary = grouped_dict[join_pipeline_name]
        source_dictionaries = grouped_dict[source_pipeline_name]
        for source_dictionary in source_dictionaries:
            try:
                source_dictionary.update(join_dictionary[0])
                yield source_dictionary
            except IndexError:  # found no join_dictionary
                yield source_dictionary

def run(argv=None):
    """Main entry point"""
    pedestrian_data_url = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/pedestrian-counting-system-monthly-counts-per-hour/exports/json'
    location_data_url = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/pedestrian-counting-system-sensor-locations/exports/json'

    output_location = '/content/data-eng-tech-assessment/output/location_output_data.txt'


    # Download the file from `url`, save it in a temporary directory and get the path to it in the `file_name` variable:
    pedestrian_data_file, pedestrian_data_headers = urllib.request.urlretrieve(pedestrian_data_url)
    location_data_file, location_data_headers = urllib.request.urlretrieve(location_data_url)
    left_key = 'locationid'
    right_key = 'location_id'

    pipeline_options = PipelineOptions()
    p = beam.Pipeline(options=pipeline_options)

    # Read data
    pedestrian_pipeline_name = 'pedestrian_data'
    pedestrian_data = (p | 'Read pedestrian data' >> beam.io.ReadFromText(pedestrian_data_file)
                          | 'JSON_to_Dict pedestrian data' >> beam.ParDo(load_json_as_dictionary())
                      )
    location_pipeline_name = 'location_data'
    location_data = (p | 'Read location data' >> beam.io.ReadFromText(location_data_file)
                        | 'JSON_to_Dict location data' >> beam.ParDo(load_json_as_dictionary())
                        | 'Select fields location data' >> beam.ParDo(select_elements_location_data())
                    )

    pipelines_dictionary = {pedestrian_pipeline_name: pedestrian_data,
                            location_pipeline_name: location_data}
    # print("Inside run : pipelines_dictionary")
    # print(pipelines_dictionary)
    tranformation_pipeline = (pipelines_dictionary
                     | 'Left join' >> LeftJoin(
                pedestrian_pipeline_name, pedestrian_data,
                location_pipeline_name, location_data, left_key,right_key)
                    )

    output  = ( tranformation_pipeline
                | 'Cleanup' >> beam.ParDo(drop_redundant_fields())
                | 'Format Output' >> beam.Map(json.dumps)
                |beam.io.WriteToText(output_location)
              )

    result = p.run()
    result.wait_until_finish()


if __name__ == '__main__':
    run()


Data Validation

In [ ]:
import pandas as pd

In [ ]:
ped_data = pd.read_csv('https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/pedestrian-counting-system-monthly-counts-per-hour/exports/csv', delimiter = ";")
loc_data = pd.read_csv('https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/pedestrian-counting-system-sensor-locations/exports/csv', delimiter = ";")


In [ ]:
ped_data.sample(5)

,timestamp,locationid,direction_1,direction_2,total_of_directions
55964,2023-06-23T13:00:00+00:00,29,39,65,104
86964,2023-05-20T00:00:00+00:00,55,404,735,1139
57511,2023-06-24T18:00:00+00:00,77,8,1,9
114996,2023-04-01T12:00:00+00:00,37,121,201,322
41452,2023-05-13T21:00:00+00:00,87,5,4,9


In [ ]:
loc_data.sample(5)

,location_id,sensor_description,sensor_name,installation_date,note,location_type,status,direction_1,direction_2,latitude,longitude,location
78,5,Princes Bridge,PriNW_T,2009-03-26,NaN,Outdoor,A,North,South,-37.818742,144.967877,"-37.81874249, 144.96787656"
93,68,Flinders Ln -Degraves St (North),FLDegN_T,2020-06-03,NaN,Outdoor,A,East,West,-37.816848,144.965598,"-37.8168479, 144.96559789"
69,93,East Melbourne Library,EastLib_T,2013-02-02,NaN,Indoor,A,In,Out,-37.814984,144.986388,"-37.81498411, 144.98638807"
89,56,Lonsdale St - Elizabeth St (North),Lon364_T,2018-07-25,NaN,Outdoor,A,East,West,-37.812348,144.961533,"-37.81234775, 144.96153311"
0,2,Bourke Street Mall (South),Bou283_T,2009-03-30,NaN,Outdoor,A,East,West,-37.813807,144.965167,"-37.81380668, 144.96516718"


In [ ]:
out_data = pd.read_json('/content/data-eng-tech-assessment/output/location_output_data.txt-00000-of-00001', lines=True)
out_data.sample(5)

,timestamp,locationid,direction_1,direction_2,total_of_directions,sensor_description,sensor_name
80648,2023-05-22 11:00:00+00:00,28,69,337,406,The Arts Centre,VAC_T
41690,2023-06-07 01:00:00+00:00,58,546,177,723,Bourke St - Spencer St (North),Bou688_T
68941,2023-05-21 20:00:00+00:00,17,94,43,137,Collins Place (South),Col15_T
144337,2023-06-25 20:00:00+00:00,67,28,46,74,Flinders Ln -Degraves St (South),FLDegS_T
52382,2023-05-26 07:00:00+00:00,79,650,517,1167,Flinders St (South),FliSS_T


In [ ]:
ped_data[ped_data.locationid == 49]['locationid'].count() == out_data[out_data.locationid == 49]['locationid'].count()

True

In [ ]:
loc_data[loc_data.location_id == 49][['location_id','sensor_description']].drop_duplicates().reset_index(drop=True)

,location_id,sensor_description
0,49,QVM-Therry St (South)


In [ ]:
out_data[out_data.locationid == 49][['locationid','sensor_description']].drop_duplicates().reset_index(drop=True)

,locationid,sensor_description
0,49,QVM-Therry St (South)
